In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(font_scale=1.4)
fontsize=15
bold="bold"

# **Preprocessing data**

In [ ]:
framingham = pd.read_csv("../input/heart-disease-prediction-using-logistic-regression/framingham.csv")
framingham

In [ ]:
#checking correlation
corr = framingham.corr()
matrix = np.triu(corr)

plt.figure(figsize=(18, 10))
sns.heatmap(data=corr, annot=True, mask=matrix, cmap=plt.cm.Blues)
plt.show()

In [ ]:
framingham.info()

In [ ]:
framingham.TenYearCHD.value_counts()

In [ ]:
framingham.isna().sum()

In [ ]:
framingham.dropna(inplace=True)
framingham.reset_index(drop=True, inplace=True)

In [ ]:
framingham

In [ ]:
framingham.isna().sum()

# Model Building

### **Get X, y data from dataframe**

In [ ]:
X = framingham.iloc[:, :-1].values
y = framingham.iloc[:, -1].values
print(X.shape,"\n",y.shape)

### **Splitting trainset, testset**

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=1)
print(Xtrain.shape, "\n", ytrain.shape, "\n", Xtest.shape)

### **Scaling data**

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.fit_transform(Xtest)

### **Training with defaults parameter**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

classifier = LogisticRegression(random_state=0)
classifier.fit(Xtrain, ytrain)
y_pred = classifier.predict(Xtest)

### **Simple evaluation**

In [ ]:
cm = confusion_matrix(ytest, y_pred)
accuracy = accuracy_score(ytest, y_pred)
print("Confusition_matrix \n", cm, "\n \nAccuracy", accuracy*100)

#### **Confusition matrix**
#### + Có 773 true positive (Số người dự đoán chính xác có bệnh)
#### + Có 11 true negative (Số người được dự đoán chính xác không có bệnh)

In [ ]:
label_0 = framingham.loc[framingham.TenYearCHD == 0,:]
label_1 = framingham.loc[framingham.TenYearCHD == 1,:]

In [ ]:
label_1.shape[0]/label_0.shape[0]

### **Đánh giá cụ thể hơn trên Confusion Matrix**

In [ ]:
from sklearn.metrics import plot_confusion_matrix

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
ax1.grid(False)
ax2.grid(False)
arguments = {"estimator": classifier, "X": Xtest, "y_true": ytest, "cmap": plt.cm.Blues}
disp1 = plot_confusion_matrix(**arguments, ax=ax1)
disp2 = plot_confusion_matrix(**arguments, ax=ax2, normalize="true")
plt.show()

| **<h3>Giá trị trên confusion maxtrix</h3>**                        | **<h3>Tỉ lệ trên confusion matrix</h3>**
|--------------------------------------------------------------------|----------------------------------
|**TP** = 11, có 11 người được dự đoán bị bệnh là chính xác.         |**TPR** = TP/(TP + FN)  , Tỉ lệ người được dự đoán có bệnh trên tổng số người có bệnh thực tế. (lớn tốt)
|**FP** = 7, có 7 người được dự đoán bị bệnh là sai.                 |**FPR** = FP/(FP + TN)  , Tỉ lệ người được dự đoán có bệnh nhưng thực tế không có bệnh trên tổng số người không có bệnh. (nhỏ tốt)
|**TN** = 773, có 773 người được dự đoán không bị bệnh là chính xác. |**TNR** = TN/(TN + FP)  , Tỉ lệ người được dự đoán không có bệnh trên tổng số người thực tế ko có bệnh. (lớn tốt)
|**FN** = 123, có 123 người được dự đoán không bị bệnh là sai.       |**FNR** = FN/(FN + TP)  , Tỉ lệ người được dự đoán không có bệnh nhưng sai thực tế trên tổng số ngừoi có bệnh thực tế. (nhỏ tốt)

### **==> Kết luận:** *Chỉ số false càng nhỏ càng tốt, chỉ số true càng lớn càng tốt.*

### **Precision Recall**

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.grid(False)
ax.set_title("Confusion Matrix", fontsize=fontsize, fontweight=bold)
plot_confusion_matrix(classifier, Xtest, ytest, ax=ax, cmap=plt.cm.Blues)
plt.show()

precision = 11 / (11 + 7) = 61%

recall = 11 / (123 + 11) = 8.2%

### **Đánh giá mô hình bằng Precision và Recall**

#### **Precision:** Tỷ lệ dự đoán đúng trên tổng số dự đoán của Positive.(Người có bệnh)
#### **Recall:** Tỷ lệ dự đoán đúng trên tổng số người bệnh thực tế. (Cũng là TPR)

### **Wrap Up:** *2 tỷ lệ trên đều nằm trong khoảng [0:1], chúng nên được cân bằng với nhau vì nếu một trong 2 tỷ lệ nhỏ hơn nhiều so với cái còn lại thì mô hình không có ý nghĩa*

#### **Ex:** 
#### + Nếu Precision cao nghĩa là số người mình dự đoán bị bệnh đúng nhiều, khi đó nếu Recall thấp thì nghĩa là mình chỉ dự đoán đúng phần nhỏ số người bệnh trên thực tế.
#### + Nếu Recall cao nghĩa là đã dự chính xác số người bệnh trên thực tế cao, khi đó nếu Precision thấp suy ra mô hình đã dự đoán nhầm rất nhiều người không bị bệnh thành bị bệnh. Mô hình trở nên vô nghĩa. Kiểu như câu nói "Em tin chắc cả thế giới này cũng có người yêu anh." 

In [ ]:
from sklearn.metrics import average_precision_score
# Tính precision trung bình qua các ngưỡng

y_score = classifier.decision_function(Xtest) # Cụ này tính khoảng cách giữa siêu phẳng của lớp kết quả và điểm data được đưa vào.
average_precision = average_precision_score(ytest, y_score, pos_label=1)

#### *Công thức*
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>AP</mtext>
  <mo>=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mi>n</mi>
  </munder>
  <mo stretchy="false">(</mo>
  <msub>
    <mi>R</mi>
    <mi>n</mi>
  </msub>
  <mo>&#x2212;</mo>
  <msub>
    <mi>R</mi>
    <mrow data-mjx-texclass="ORD">
      <mi>n</mi>
      <mo>&#x2212;</mo>
      <mn>1</mn>
    </mrow>
  </msub>
  <mo stretchy="false">)</mo>
  <msub>
    <mi>P</mi>
    <mi>n</mi>
  </msub>
</math>

In [ ]:
from sklearn.metrics import plot_precision_recall_curve
# Compute precision-recall pairs for different probability thresholds and plot.

fig, ax = plt.subplots(figsize=(15, 7))
disp = plot_precision_recall_curve(classifier, Xtest, ytest, color="navy", ax=ax)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision), fontweight=bold)
disp.ax_.grid(True)
disp.ax_.legend(fontsize=16)
plt.show()

### **Giá trị AP (Average Precision)**

#### - AP là một giá trị đánh giá chung đường của chúng ta. (Nó giống AUC của ROC)
#### - AP càng gần 1 thì Precision và Recall càng ổn định.
#### - Có thể hiểu AP như độ chính xác trung bình của mô hình. 

### Đánh giá mô hình trên ROC_AUC

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_curve, plot_roc_curve


fig, ax = plt.subplots(figsize=(10, 7))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# fpr, tpr, _ = roc_curve(ytest, y_score, pos_label=1)
# roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax)
disp = plot_roc_curve(classifier, Xtest, ytest, color="darkgreen", ax=ax)
# remembering asign pos_label for plot
disp.ax_.grid(True)
disp.ax_.legend(fontsize=15)
disp.ax_.set_title("sự thay đổi của TPR, FPR theo các sự tăng dần ngưỡng", fontweight="bold")
plt.show()

#### Biểu đồ trên có AUC = 0.72 có ý nghĩa là mô hình có thể phần loại được Positive và Negative nhưng vẫn còn lỗi (FP, FN).

### **Các mức ý nghĩa của AUC:**
#### + *AUC = 1*: Mô hình phân loại rạch ròi giữa 2 lớp. Không có lỗi (FN, FP).
#### + *AUC = 0.5*: Trường hợp tệ nhất, mô hình không phân biệt được 2 lớp.
#### + *AUC ~ 0*: Mô hình phân loại ngược.

### **Kết luận**
#### + Dựa vào biểu đồ ta có thể chọn ngưỡng phù hợp cho bài toán.(TPR, FPR)
#### + AUC giúp ta xác điệu hiệu suất mô hình.

### **<span style="color:green"> When to Use ROC vs. Precision-Recall Curves? </span>**

#### + ROC curves nên dùng khi số lượng nhãn giữa các lớp gần như bằng nhau.
#### + Precision-Recall curves nên dùng khi số lượng nhãn giữa các lớp lệch từ trung bình đến lệch cao.